# 8.3 使用预训练模型

解决小型数据集问题,一个非常重要的做法是使用预训练模型加以调整.

预训练模型通常是在一个非常大而全的原始数据集上训练,那么模型的深层信息有可能是对视觉世界的通用模型.例如一个 imageNet 上训练的模型(动物/日常物品)可用用来识别家具物品等完全不同的问题.这样的深层可移植 可复制,是深度学习对之前浅度学习的一大优势.

这里选择的是在 imageNet 上训练的 cnn 模型,imageNet 包含 140 万张标记的图片和 1000 多给标签,其中也包含了许多不同种类的猫/狗,我们是可以期待使用 imageNet 训练出的模型效果.

本章使用的 VGG16 模型,是 2014 年 Karen Simonyan 和 Andrew Zisserman 开发的.虽然截止到今日,VGG16 是比较旧的模型,其效果不仅无法和最新的模型相比,VGG16 还要更加复杂.但是 VGG16 的架构和我们依旧熟悉的架构非常相似,方便我们理解.

像是 VGG, ResNet, Inception, Xception 这样的模型名称,我们还好见到很多很多..

预训练的网络有两者使用方式,我们都会见到

- 特征提取
- 微调模型
